In [1]:
from pathlib import Path
import torch
from datatype_recovery.models.dataset import load_dataset_from_path


model_path = Path.home()/'trained_models/structural_4out_4hops_comp.pt'
dataset_path = Path.home()/'trainset_astera_5hops_nocomp'
dataset = load_dataset_from_path(dataset_path)

model = torch.load(model_path)

In [2]:
from torch_geometric.loader import DataLoader
from datatype_recovery.models.dataset.encoding import *

transform = ToFixedLengthTypeSeq(fixed_len=4)
dataset.transform = transform

# encode_typeseq(['<EMPTY>']*num_empty_slots)
# extended = torch.cat((dataset[1].y, encode_typeseq(['<EMPTY>']*num_empty_slots)), dim=2)
# decode_typeseq(dataset[1].y)
# decode_typeseq(extended)
print(decode_typeseq(dataset[12].y))
# dataset.transform = None

decode_typeseq(dataset[1].y)
dataset[1].y.shape

['PTR', 'STRUCT', '<EMPTY>', '<EMPTY>']


torch.Size([1, 22, 4])

In [3]:
model.eval()

loader = DataLoader(dataset, batch_size=2)

iterable = loader.__iter__()
# iterable = iter(loader)
data = next(iterable)
# data.y.argmax(dim=1)
# data = next(iterable)
# data = next(iterable)
# data = next(iterable)
# data = next(iterable)
# data.y.T.argmax(dim=0)
# data.y.T.argmax(dim=0)
# data.y.T[None,:,:]

# decode_typeseq(encode_typeseq(decode_typeseq(data.y)))
print(data.y[0].shape)
print(decode_typeseq(data.y[0]))
# torch.cat(x for x in data.y)
decode_typeseq(data.y[0][None,:,:])
data.y[0].argmax(dim=data.y[0].dim()-2)
# decode_typeseq(data.y)
for yy in data.y:
    print(decode_typeseq(yy))

torch.Size([22, 4])
['ARR', 'uchar', '<EMPTY>', '<EMPTY>']
['ARR', 'uchar', '<EMPTY>', '<EMPTY>']
['int32', '<EMPTY>', '<EMPTY>', '<EMPTY>']


In [4]:


out = model(data.x, data.edge_index, data.batch)


print(out.shape)
for pred in out.argmax(dim=1):
    print(pred)
    print([type_seq_names[i] for i in pred])

out.argmax(dim=1)
out.max(dim=1).values
out[0]

torch.Size([2, 22, 4])
tensor([12, 16, 16, 16])
['uint128', 'STRUCT', 'STRUCT', 'STRUCT']
tensor([12, 16, 16, 16])
['uint128', 'STRUCT', 'STRUCT', 'STRUCT']


tensor([[-0.0572, -0.0158, -0.0904,  0.0697],
        [-0.0031,  0.0420,  0.0409, -0.0602],
        [-0.0453,  0.0579,  0.0026, -0.0612],
        [ 0.0658, -0.0566, -0.0207,  0.0261],
        [ 0.0212,  0.0070,  0.0036,  0.0525],
        [ 0.0640, -0.0095, -0.0410,  0.0605],
        [-0.0456,  0.0476,  0.0548, -0.0082],
        [-0.0230,  0.0271,  0.0400, -0.0312],
        [-0.0829,  0.0594,  0.0160, -0.0709],
        [ 0.0545, -0.0064, -0.0474,  0.0629],
        [-0.0919, -0.1007, -0.0173, -0.0193],
        [-0.0032,  0.0198, -0.0029, -0.0310],
        [ 0.0822, -0.0373, -0.0696,  0.0583],
        [ 0.0337, -0.0559,  0.0562,  0.0350],
        [ 0.0223, -0.0642,  0.0329, -0.0584],
        [-0.0308,  0.0056, -0.0787, -0.0398],
        [-0.0585,  0.0931,  0.0767,  0.0841],
        [-0.0352, -0.0783, -0.0459, -0.0302],
        [ 0.0491, -0.0007, -0.0167,  0.0202],
        [-0.0506, -0.0323, -0.0780, -0.0264],
        [-0.0840,  0.0506, -0.0590, -0.0418],
        [-0.0400,  0.0498, -0.0184

In [5]:
# Link: https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
# TODO: I think per link above we need to return a 3d tensor from our model
# of shape: (batch_size, num_classes, [max_]seq_len)
#
# - while it seems to be working as-is (and the math probably works out since I'm
#   doing things consistently "wrong" lol), it would be better to just use the
#   expected dimensionality. Not only for CrossEntropy itself, but downstream functions
#   now do NOT need to know the max_seq_len - just take the final dimension (# cols)

# TODO: change the model to return this instead
# --> also update the decode funcs etc...
# return logits.view((batch_size, self.num_classes, self.max_seq_len))

In [6]:
# TODO: - all this logic needs to end up in encoding.py
all_names = type_seq_names.copy()

nonterminals = ['ARR','PTR']    # <EMPTY>?

INCLUDE_COMP = False
INCLUDE_EMPTY = False

special = []
if INCLUDE_EMPTY:
    special.append('<EMPTY>')
else:
    all_names.remove('<EMPTY>')

if INCLUDE_COMP:
    special.append('COMP')
else:
    all_names.remove('COMP')

aggregates = ['STRUCT', 'UNION', 'FUNC', *nonterminals]

non_primitives = [*aggregates, *special, *nonterminals]
primitives = [x for x in all_names if x not in non_primitives]
terminals = [x for x in all_names if x not in nonterminals]

In [7]:

terminals

['float',
 'double',
 'long double',
 'char',
 'short',
 'int32',
 'int64',
 'int128',
 'uchar',
 'ushort',
 'uint32',
 'uint64',
 'uint128',
 'void',
 'STRUCT',
 'UNION',
 'ENUM',
 'FUNC']

In [8]:
nonterminals

['ARR', 'PTR']

In [9]:
non_primitives

['STRUCT', 'UNION', 'FUNC', 'ARR', 'PTR', 'ARR', 'PTR']

In [10]:
aggregates

['STRUCT', 'UNION', 'FUNC', 'ARR', 'PTR']

In [11]:
special

[]

In [12]:
terminals

['float',
 'double',
 'long double',
 'char',
 'short',
 'int32',
 'int64',
 'int128',
 'uchar',
 'ushort',
 'uint32',
 'uint64',
 'uint128',
 'void',
 'STRUCT',
 'UNION',
 'ENUM',
 'FUNC']

In [13]:
# 1-level
nonterminals
single_level_count = len(terminals) + len(nonterminals)
single_level_count
len(all_names)

20

In [49]:
def get_all_possible_classes_for_seq_len(seq_len:int) -> List[List[str]]:
    all_classes = []
    prev_nts = []

    for level_idx in range(seq_len):
        current_level = [[x] for x in all_names]
        if prev_nts:
            for pnt in prev_nts:
                # add this level to nonterminals from prev level
                all_classes.extend([[*pnt, *x] for x in current_level])
        else:
            all_classes = current_level

        # current_nts = [x for x in all_classes if x[-1] in nonterminals and len(x) == level_idx+1]
        # current_nts = list(filter(lambda x: x[-1] in nonterminals and len(x) == level_idx+1, all_classes))
        current_nts = []
        for x in all_classes:
            if x[-1] in nonterminals and len(x) == level_idx+1:
                current_nts.append(x.copy())
        prev_nts = current_nts

    return all_classes

len(get_all_possible_classes_for_seq_len(1))

TypeError: object of type 'NoneType' has no len()

In [36]:
for i in range(3):
    print(type(i))

<class 'int'>
<class 'int'>
<class 'int'>


In [30]:
all_classes

[['float'],
 ['double'],
 ['long double'],
 ['char'],
 ['short'],
 ['int32'],
 ['int64'],
 ['int128'],
 ['uchar'],
 ['ushort'],
 ['uint32'],
 ['uint64'],
 ['uint128'],
 ['void'],
 ['STRUCT'],
 ['UNION'],
 ['ENUM'],
 ['FUNC']]

In [14]:
second_level_classes = []
for x in nonterminals:
    for y in all_names:
        second_level_classes.append([x, y])
second_level_classes

[['ARR', 'float'],
 ['ARR', 'double'],
 ['ARR', 'long double'],
 ['ARR', 'char'],
 ['ARR', 'short'],
 ['ARR', 'int32'],
 ['ARR', 'int64'],
 ['ARR', 'int128'],
 ['ARR', 'uchar'],
 ['ARR', 'ushort'],
 ['ARR', 'uint32'],
 ['ARR', 'uint64'],
 ['ARR', 'uint128'],
 ['ARR', 'void'],
 ['ARR', 'PTR'],
 ['ARR', 'ARR'],
 ['ARR', 'STRUCT'],
 ['ARR', 'UNION'],
 ['ARR', 'ENUM'],
 ['ARR', 'FUNC'],
 ['PTR', 'float'],
 ['PTR', 'double'],
 ['PTR', 'long double'],
 ['PTR', 'char'],
 ['PTR', 'short'],
 ['PTR', 'int32'],
 ['PTR', 'int64'],
 ['PTR', 'int128'],
 ['PTR', 'uchar'],
 ['PTR', 'ushort'],
 ['PTR', 'uint32'],
 ['PTR', 'uint64'],
 ['PTR', 'uint128'],
 ['PTR', 'void'],
 ['PTR', 'PTR'],
 ['PTR', 'ARR'],
 ['PTR', 'STRUCT'],
 ['PTR', 'UNION'],
 ['PTR', 'ENUM'],
 ['PTR', 'FUNC']]

In [15]:
second_level_nonterminals = [x for x in second_level_classes if x[-1] not in terminals]
second_level_nonterminals

[['ARR', 'PTR'], ['ARR', 'ARR'], ['PTR', 'PTR'], ['PTR', 'ARR']]

In [16]:
third_level_classes = []
for sln in second_level_nonterminals:
    for x in all_names:
        third_level_classes.append([*sln, x])
third_level_classes

[['ARR', 'PTR', 'float'],
 ['ARR', 'PTR', 'double'],
 ['ARR', 'PTR', 'long double'],
 ['ARR', 'PTR', 'char'],
 ['ARR', 'PTR', 'short'],
 ['ARR', 'PTR', 'int32'],
 ['ARR', 'PTR', 'int64'],
 ['ARR', 'PTR', 'int128'],
 ['ARR', 'PTR', 'uchar'],
 ['ARR', 'PTR', 'ushort'],
 ['ARR', 'PTR', 'uint32'],
 ['ARR', 'PTR', 'uint64'],
 ['ARR', 'PTR', 'uint128'],
 ['ARR', 'PTR', 'void'],
 ['ARR', 'PTR', 'PTR'],
 ['ARR', 'PTR', 'ARR'],
 ['ARR', 'PTR', 'STRUCT'],
 ['ARR', 'PTR', 'UNION'],
 ['ARR', 'PTR', 'ENUM'],
 ['ARR', 'PTR', 'FUNC'],
 ['ARR', 'ARR', 'float'],
 ['ARR', 'ARR', 'double'],
 ['ARR', 'ARR', 'long double'],
 ['ARR', 'ARR', 'char'],
 ['ARR', 'ARR', 'short'],
 ['ARR', 'ARR', 'int32'],
 ['ARR', 'ARR', 'int64'],
 ['ARR', 'ARR', 'int128'],
 ['ARR', 'ARR', 'uchar'],
 ['ARR', 'ARR', 'ushort'],
 ['ARR', 'ARR', 'uint32'],
 ['ARR', 'ARR', 'uint64'],
 ['ARR', 'ARR', 'uint128'],
 ['ARR', 'ARR', 'void'],
 ['ARR', 'ARR', 'PTR'],
 ['ARR', 'ARR', 'ARR'],
 ['ARR', 'ARR', 'STRUCT'],
 ['ARR', 'ARR', 'UNION']

In [20]:
[x for x in third_level_classes if x[-1] not in terminals]

[['ARR', 'PTR', 'PTR'],
 ['ARR', 'PTR', 'ARR'],
 ['ARR', 'ARR', 'PTR'],
 ['ARR', 'ARR', 'ARR'],
 ['PTR', 'PTR', 'PTR'],
 ['PTR', 'PTR', 'ARR'],
 ['PTR', 'ARR', 'PTR'],
 ['PTR', 'ARR', 'ARR']]

In [17]:
len(all_names)

20

In [18]:
len(second_level_classes)

40

In [19]:
len(third_level_classes)

80